In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
df = pd.read_csv('z_spotify_dataset.csv', 
                         skiprows=1,
                         names=['user_id', 'artistname', 'trackname', 'playlistname'],
                         on_bad_lines='skip')
df

,user_id,artistname,trackname,playlistname
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010
...,...,...,...,...
12891675,2302bf9c64dc63d88a750215ed187f2c,Mötley Crüe,Wild Side,iPhone
12891676,2302bf9c64dc63d88a750215ed187f2c,John Lennon,Woman,iPhone
12891677,2302bf9c64dc63d88a750215ed187f2c,Tom Petty,You Don't Know How It Feels,iPhone
12891678,2302bf9c64dc63d88a750215ed187f2c,Tom Petty,You Wreck Me,iPhone


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12891680 entries, 0 to 12891679
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   user_id       object
 1   artistname    object
 2   trackname     object
 3   playlistname  object
dtypes: object(4)
memory usage: 393.4+ MB


Данные содержат исключительно категориальные значения

In [12]:
df.describe()

,user_id,artistname,trackname,playlistname
count,12891680,12858108,12891592,12890434
unique,15918,289820,2032043,157504
top,4398de6902abde3351347b048fcdc287,Daft Punk,Intro,Starred
freq,295275,36086,6676,1337085


Колонка user_id заполнена полностью, остальные имеют пробелы
Проверим это:

In [13]:
df.isnull().sum()

user_id             0
artistname      33572
trackname          88
playlistname     1246
dtype: int64

В колонке artistname есть пустые значения. Учитывая, что рекомендательная система должна быть построена в первую очередь на рекомендациях по исполнителю и треку, то данные строки нужно исключить из дата сета. Такие строки, от которых зависит рекомендация, должны быть обязательно заполнены, иначе они будут создавать лишний шум.
После исключения проверим вновь, останутся ли пропуски среди других колонок

In [14]:
df_clean = df.dropna(subset=['artistname', 'trackname'])

In [15]:
df_clean.isnull().sum()

user_id            0
artistname         0
trackname          0
playlistname    1246
dtype: int64

In [9]:
df_clean.shape

(12858077, 4)

Колонка playlistname осталась без изменений - 1246 строк не имеют наименования плейлиста. На текущий момент, оставляю эти строки в списке.

In [16]:
# Посмотрим наполнение плейлистов для каждого пользователя
top_playlistname = df_clean.groupby(['user_id', 'playlistname']).count().sort_values('artistname', ascending = False)
top_playlistname
# Заметны повторения названий плейлистов

,,artistname,trackname
user_id,playlistname,,
4398de6902abde3351347b048fcdc287,Starred,47362,47362
68756b4d60b6fac27789e85c2b63d1c6,Starred,18509,18509
c410ee930bd5f0e4893632503ec17b05,Starred,18081,18081
f2604a69836c94d00fcd11dd602944fc,everything,15213,15213
6b85a8076b4f3b1036ef15f09e2feeeb,Generos,14854,14854
...,...,...,...
63dad8e72eeb1179dd21f279b5a4c603,wiz kid In My Bed / afro,1,1
63e943a5e78a0d23eb1271d2ea0c97af,Becoming the Gunship,1,1
087f0f8bc5eeab5b2dc81f4ce200fd4e,All Or Nothing,1,1


In [17]:
# сделаем сводку плейлистов и проверим количество уникальных пользователей для каждого названия
df_clean.groupby('playlistname').nunique().sort_values('user_id', ascending = False)

,user_id,artistname,trackname
playlistname,,,
Starred,5016,76673,437368
Liked from Radio,3125,19602,69932
Favoritas de la radio,745,5960,17018
My Shazam Tracks,424,5228,8093
Christmas,312,3048,6958
...,...,...,...
"37 no son nada""",1,1,1
538 Hitzone - Best Of 2012,1,44,45
80's,1,22,32


Благодаря таблицам выше мы видим, что в дата сете присутствуют повторы названий плейлистов у разных пользователей. Следовательно, можно сделать вывод, что обучать модель необходимо на следующих признаках: user_id, artistname, trackname. В виду того, что у каждого пользователя может быть свое видение что ему понравилось, например на радио ("Liked from Radio" - 3125 пользователей сделали твиты с таким названием), то и обучить модель на таких разных данных не корректно. Лучше исходить из схожести интересов пользователей. 

In [18]:
# Проверим, на сколько пользователи активны в составлении плейлистов
df_clean.groupby('user_id').nunique().sort_values('playlistname', ascending = False)

# Как видно из таблицы ниже, есть пользователи, которые составили несколько плейлистов, а есть те, кто составил только один

,artistname,trackname,playlistname
user_id,,,
872a19d491f4357408551be6d7eea273,617,4086,184
19ccdb5f2ce9193e6028923f65afaac2,327,1196,105
57c4979b17f7563f83e69f09701eb584,71,878,76
39bd43df0c61c451361200312a4ab6e0,672,3644,74
13c6654d04eb2e0c09671c8868393722,139,1017,68
...,...,...,...
c542bad3598f9dd7e338722080625f94,34,38,1
c52fd58d8a570688a984d2ed64c065fe,1,15,1
0059ac6b3ba3c3f415f09059a3bd703a,15,167,1


In [19]:
# Посмотрим на количество треков, которые добавил к себе пользователь
full_user = df_clean.groupby('user_id')[['trackname']].count().sort_values('trackname', ascending = False).reset_index()
full_user

,user_id,trackname
0,4398de6902abde3351347b048fcdc287,295274
1,7ee2b92c5bcf6133b8132363e5bda960,90630
2,99deafd9b792af8e6a535483088faef2,65909
3,fa849dabeb14a2800ad5130907fc5018,60713
4,6b85a8076b4f3b1036ef15f09e2feeeb,39592
...,...,...
15909,8c7db5f1570e8149015083f16e671e03,1
15910,8c9d6a258669fab92272b825ad19f308,1
15911,8cb4f5647b0616e63ca0bc6c3eea24a1,1
15912,5dc479e1dd91b954f30bee701decf911,1


In [66]:
df_clean.shape

(12858077, 4)

В дата сете присутствуют пользователи, которые "очень активные" и "не очень активные". Проверим, сколько пользователей добавили к себе более 1000, от 1000 до 101, от 100 до 11 и менее 11 песен.

In [20]:
# количество пользователей, у которых количество треков менее и равно 10
print("Треков менее и равно 10: ", full_user[full_user.trackname <= 10].count())

# количество пользователей, у которых количество треков менее 101, но более 10
print("Треков менее 100, но более 10: ", full_user[(full_user.trackname <= 100) & (full_user.trackname > 10)].count())

# количество пользователей, у которых количество треков менее 1001, но более 101
print("Треков менее 999, но более 101: ", full_user[(full_user.trackname <= 999) & (full_user.trackname > 100)].count())

# количество пользователей, у которых количество треков 1000 и более
print("Треков 1000 и более: ", full_user[full_user.trackname > 999].count())

Треков менее и равно 10:  user_id      1013
trackname    1013
dtype: int64
Треков менее 100, но более 10:  user_id      3172
trackname    3172
dtype: int64
Треков менее 999, но более 101:  user_id      8334
trackname    8334
dtype: int64
Треков 1000 и более:  user_id      3395
trackname    3395
dtype: int64


Из общего количества пользователей - 15 893, около 26,3% пользователей имеют в своем активе 100 и менее песен. Остальная же часть (73.7%) являются "более активными пользователями" и имеют сравнительно большее количество песен.
В связи с этим, 26,3% пользователей будут исключены из дата сета, т.к. наличие "более активных пользователей" может привести к ухудшению результатов в обучении модели.

In [21]:
# Вывожу всех пользователей, у которых треков 100 и менее в отдельный список
lst_user = full_user[full_user.trackname <= 100]['user_id'].tolist()

In [22]:
# перезаписываю датафрейм, в котором исключаю всех пользователей у котрых количество треков 100 и менее
df_clean = df_clean[df_clean.user_id.isin(lst_user) == False]

In [23]:
df_clean.groupby('user_id')[['trackname']].count().sort_values('trackname', ascending = False).reset_index()

,user_id,trackname
0,4398de6902abde3351347b048fcdc287,295274
1,7ee2b92c5bcf6133b8132363e5bda960,90630
2,99deafd9b792af8e6a535483088faef2,65909
3,fa849dabeb14a2800ad5130907fc5018,60713
4,6b85a8076b4f3b1036ef15f09e2feeeb,39592
...,...,...
11724,1bdace870bc8569fdf0dca8497d637dd,101
11725,673cb01c98ae6d9f91e0334045effb7a,101
11726,1b51b1b406f7dceeeab4c3eff428ed64,101
11727,c929ca67a2f4e9d2b1462b09055a2ba9,101


In [71]:
df_clean.shape

(12706733, 4)

In [24]:
# Посмотрим наиболее полулярного артиста среди пользователей
top_artists = df_clean.groupby('artistname').count().sort_values('user_id', ascending=False).reset_index()
top_artists

# Общая сводка по всем пользователям, по всем плейлистам с учетом повторений наименования артиста

,artistname,user_id,trackname,playlistname
0,Daft Punk,35452,35452,35452
1,Coldplay,34805,34805,34805
2,Radiohead,31010,31010,31010
3,The Rolling Stones,30573,30573,30555
4,Kanye West,28671,28671,28671
...,...,...,...,...
287747,﻿Butch,1,1,1
287748,﻿Lil Jon,1,1,1
287749,＀﹚edd feat. Matthew Koma,1,1,1
287750,ＡＫＩＮＯ,1,1,1


Исполнителя Daft Punk пользователи добавляли чаще всего в плейлисты

In [25]:
# Сводка артистов и количество уникальных пользователей, которым интересно творчество и количество их композиций, которые были добавлены в плейлисты
top_artists_nunique = df_clean.groupby('artistname').nunique().sort_values('user_id', ascending=False).reset_index()
top_artists_nunique

,artistname,user_id,trackname,playlistname
0,Daft Punk,4388,641,5590
1,Coldplay,4328,466,6119
2,Rihanna,3806,666,5572
3,David Guetta,3573,704,5018
4,JAY Z,3465,585,4439
...,...,...,...,...
287747,#1 Katy Perry Tribute Band,1,2,1
287748,티아라(Tiara),1,1,1
287749,티아라 (T-ara),1,1,1
287750,#1 Charity Songs,1,1,1


Однако, степень популярности исполнителя более показательна по тому, насколько чаще добавляют трек исполнителя к себе в плейлист. Поэтому будем работать с таблицей общей сводки артистов.

In [26]:
# Посмотрим, у скольких артистов пользователи добавили треки 10 и менее, 11 - 100, более 100

print("Добавляли 10 и менее треков: ", top_artists['user_id'][top_artists.user_id <= 10].count())
print("Добавляли 11 - 100: ", top_artists['user_id'][(top_artists.user_id > 10) & (top_artists.user_id <= 100)].count())
print("Добавляли более 100: ", top_artists['user_id'][top_artists.user_id > 100].count())

Добавляли 10 и менее треков:  219650
Добавляли 11 - 100:  54060
Добавляли более 100:  14042


Как видим, у подавляющего числа исполнителей добавляли треки 10 и менее раз. Следовательно, исполнители малоизвестны и менее популярны. В данном случае предполагается отказаться от этих исполнителей, т.к. обычно, при рекомендации - пользователь более доволен подборкой из наиболее популярных треков.

In [27]:
# Записываю в лист всех исполнителей, которых добавляли менее 11 раз
lst_artist = top_artists['artistname'][top_artists.user_id <= 10].tolist()

In [28]:
# Удаляем строки с артистами, которых добавляли менее 11 раз
df_clean = df_clean[df_clean.artistname.isin(lst_artist) == False]


In [77]:
df_clean.shape

(12153924, 4)

In [29]:
# Сколько артистов имеют не более 3 песен
tree_song = df_clean.groupby('artistname').nunique().sort_values('trackname', ascending=False).reset_index()
tree_song[tree_song.trackname <= 3]['trackname'].count()

np.int64(3539)

Данные исполнители весьма не показательны. Если есть или просто найдены не более 3 треков - говорит о том, что исполнитель не на столько известен

In [30]:
lst_tree_song = tree_song[tree_song.trackname <= 3]['artistname'].tolist()

In [31]:
# Удаляем строки с исполнителями, которые имеют только 3 добавленные песни
df_clean = df_clean[df_clean.artistname.isin(lst_tree_song) == False]

In [82]:
df_clean.shape

(12017852, 4)

In [20]:
# # Посмотрим наиболее популярный трек среди пользователей
# top_song = df_clean.groupby(['artistname','trackname']).count().sort_values('user_id', ascending=False).reset_index()
# top_song

,artistname,trackname,user_id,playlistname
0,M83,Midnight City,2609,2609
1,Daft Punk,Get Lucky - Radio Edit,2341,2341
2,Imagine Dragons,Radioactive,2336,2336
3,Of Monsters and Men,Little Talks,2254,2254
4,Avicii,Wake Me Up,2242,2242
...,...,...,...,...
2367589,아마츄어 증폭기,프롤레타리아트,1,1
2367590,!!!,Broadway [Gimme The Lights On] (Romare Mix),1,1
2367591,아마츄어 증폭기,사소설,1,1
2367592,아마츄어 증폭기,산양,1,1


<!-- Сделаем градацию по добавлению пользователями песен: 1 песня, 2-10, 10-100, свыше 100 -->

In [21]:
# print('Песню добавил только 1 пользователь: ', top_song[top_song.user_id == 1]['user_id'].count())
# print('Песню добавили 2-10 пользователей: ', top_song[(top_song.user_id > 1) & (top_song.user_id <= 100)]['user_id'].count())
# print('Песню добавили свыше 100 пользователей: ', top_song[top_song.user_id > 100]['user_id'].count())

Песню добавил только 1 пользователь:  1238626
Песню добавили 2-10 пользователей:  1115980
Песню добавили свыше 100 пользователей:  12988


На основе отчищенных от "шумов" данных создаем матрицу соответствия. Строками будут являтся пользователи, колонками исполнители, в соответствующей клетке пересечения будет накапливаться сумма треков добавленных пользователем соответствующего исполнителя. Для приведения матрицы к содержанию только нулей и единиц - разделим матрицу на матрицу 

In [32]:
df_clean.to_csv('dataset_clean.csv', index=False)